## Import libraries

In [47]:
from pytwitter import Api
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import math
from pytwitter.models import User;
from pytwitter.models import Tweet;


In [6]:
api = Api(bearer_token="AAAAAAAAAAAAAAAAAAAAAMhqlAEAAAAA4Pqzn354Z5nlkP5lKaW98vzlVlA%3D7GIA03xacVKdFYTFg7qmgvWTZThpa2FFd4SNPUqP7uPK7Xjue5")

public_tweets = api.search_tweets(query="stf lang:pt has:hashtags is:retweet", expansions=["referenced_tweets.id.author_id","in_reply_to_user_id","attachments.media_keys","author_id","entities.mentions.username"], 
                                  user_fields=["created_at","entities","id","location","name","pinned_tweet_id","profile_image_url","protected","public_metrics","url","username","verified"],
                                  tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"], max_results=100)

# Analysis

## User's influence and reputation Analysis

In [30]:
def get_sentiment_score(text):
    """
    Returns the sentiment score of a given text using TextBlob.

    Parameters:
    text (str): The text to be analyzed.

    Returns:
    float: The sentiment score between -1.0 (negative) and 1.0 (positive).
    """
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [61]:
def calculate_influence_and_reputation(user: User):
    follower_count = user.public_metrics.followers_count

    # Get user's tweet count and average engagement rate
    tweets = api.get_timelines(user.id, max_results=100, tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"])
    tweet_count = len(tweets.data)
    total_engagement = 0
    
    for tweet in tweets.data:
        total_engagement += tweet.public_metrics.like_count + tweet.public_metrics.retweet_count
    if tweet_count > 0:
        avg_engagement_rate = total_engagement / (tweet_count * follower_count) if total_engagement > 0 and tweet_count > 0 and follower_count > 0 else 0
    else:
        avg_engagement_rate = 0

    # Calculate influence score
    influence_score = math.log(follower_count + 1, 10) * (avg_engagement_rate + 1)

    # Get user's recent mentions and replies
    mentions = api.search_tweets(query=f"@{user.username}", max_results=100)
    replies = api.search_tweets(query=f"to:{user.username}", max_results=100)

    # Calculate reputation score based on sentiment analysis of mentions and replies
    positive_sentiments = 0
    negative_sentiments = 0
    for mention in mentions.data:
        if mention.id != user.id:
            sentiment = get_sentiment_score(mention.text)
            if sentiment > 0:
                positive_sentiments += 1
            elif sentiment < 0:
                negative_sentiments += 1
    for reply in replies.data:
        if reply.id != user.id:
            sentiment = get_sentiment_score(reply.text)
            if sentiment > 0:
                positive_sentiments += 1
            elif sentiment < 0:
                negative_sentiments += 1
    if (positive_sentiments + negative_sentiments) > 0:
        reputation_score = positive_sentiments / (positive_sentiments + negative_sentiments)
    else:
        reputation_score = 0

    # Return influence and reputation scores
    return {'influence': influence_score, 'reputation': reputation_score }

User(id='1584531289864036353', name='@leilapsp', username='leilapsp')
{'influence': 3.2053296576880337, 'reputatioon': 0.0}
User(id='68313718', name='STF', username='STF_oficial')
{'influence': 6.438841253111726, 'reputatioon': 0.5}
User(id='1557430222953250816', name='Daniel Aversa', username='DanielAversa5')
{'influence': 16.03689688129373, 'reputatioon': 0}
User(id='49477957', name='Cláudio Humberto', username='ColunaCH')


KeyboardInterrupt: 

### Influence Calculus

### Reputation Calculus

## Network Analysis

## Text Analysis

### Text Cleaning

# Results

## Influence vs Interactions

## Reputation vs Interactions

## Histogram Interactions

## Evaluation Metrics

In [49]:
def analyze_social_capital(tweets):
    public_tweets = api.search_tweets(query="stf lang:pt has:hashtags is:retweet", expansions=["referenced_tweets.id.author_id","in_reply_to_user_id","attachments.media_keys","author_id","entities.mentions.username"],
    user_fields=["created_at","entities","id","location","name","pinned_tweet_id","profile_image_url","protected","public_metrics","url","username","verified"],
    tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"], max_results=100)

    api.get_followers()

    public_tweets.includes.tweets

#analyze_social_capital(public_tweets.data)


In [ ]:

def social_capital_impact(tweet, social_connections, interactions, texts, resources, user_popularity, user_influence):
  # extract the user, mentions, and hashtags from the tweet
  user = tweet['user']
  mentions = tweet['mentions']
  hashtags = tweet['hashtags']
  
  # compute the number of connections the user has in the social network
  user_connections = np.sum(social_connections[user])
  
  # compute the number of connections between the user and the mentions
  mention_connections = 0
  for mention in mentions:
    mention_connections += social_connections[user][mention]
  
  # compute the number of connections between the user and the hashtags
  hashtag_connections = 0
  for hashtag in hashtags:
    hashtag_connections += social_connections[user][hashtag]
  
  # compute the sum of the resources associated with the tweet
  resource_sum = 0
  for resource in resources:
    if resource in tweet['resources']:
      resource_sum += resources[resource]
  
  # compute the number of interactions for the tweet
  interaction_sum = interactions[tweet['tweet_id']]
  
  # compute the text similarity between the tweet and the other texts
  vectorizer = TfidfVectorizer()
  tweet_text = [tweet['text']]
  tweet_vector = vectorizer.fit_transform(tweet_text)
  other_vectors = vectorizer.transform(texts)
  similarity = cosine_similarity(tweet_vector, other_vectors)
  
  # compute the user popularity and influence factors
  popularity_factor = user_popularity[user]
  influence_factor = user_influence[user]
  
  # compute the social capital impact as the sum of mention connections, hashtag connections, resource sum, interaction sum,
  # text similarity, popularity factor, and influence factor divided by the number of user connections
  impact = (mention_connections + hashtag_connections + resource_sum + interaction_sum + similarity + popularity_factor + influence_factor) / user_connections
  
  return impact
